In [1]:
import lightcurves.lc_utils as lu
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#%matplotlib inline

Primero abro una curva de ejemplo y la grafico

In [2]:
# Ubicacion de las curvas
# 0-1           Be_lc
# 255-256       CEPH
# 457-458       EB
# 967-968       longperiod_lc
# 1697-1698     microlensing_lc
# 2862-2863     non_variables
# 12527-12528   quasar_lc
# 12645-12646   RRL

paths = lu.get_lightcurve_paths()
a, b = 967, 968
#a, b = 2864, 2865
#a, b = 4, 5

azul = lu.open_lightcurve(paths[a])
roja = lu.open_lightcurve(paths[b])
curva = pd.concat([azul, roja], axis=1, keys=['azul', 'roja'], join='inner')

#plt.plot( azul.index, azul['mag'], '.', color = 'b')
plt.errorbar(azul.index, azul['mag'], yerr=azul['err'], fmt=".b", ecolor='r', capsize=0)
plt.show()

In [3]:
#plt.plot( azul.index[0::10], azul['mag'].iloc[0::10], '.', color = 'b')
plt.errorbar(azul.index[0::10], azul['mag'].iloc[0::10], yerr=azul['err'][0::10], fmt=".b", ecolor='r', capsize=0)
plt.show()

### Gaussian Process

In [7]:
t_obs, y_obs, err_obs, min_time, max_time = lu.prepare_lightcurve(azul, 100)
#t_obs, y_obs, err_obs, min_time, max_time = lu.prepare_lightcurve(azul, 500)

t_obs = np.ravel(t_obs)
y_obs = np.ravel(y_obs)
err_obs = np.ravel(err_obs) 

# Reduzco a la fuerza el ruido a ver si el ajuste mejora
# err_obs = err_obs/10

In [8]:
#plt.plot( t_obs.ravel(), y_obs.ravel(), '.', color = 'b')
plt.errorbar(t_obs.ravel(), y_obs.ravel(), yerr=err_obs.ravel(), fmt=".b", ecolor='r', capsize=0)
plt.show()

In [9]:
import george
from george import kernels
from model import *

print("Fitting GP")
data = (t_obs, y_obs, err_obs)

#parameters = [m, b, alpha, l, sigma]

initial = [0.0, 0.0, -1.0, 0.1, 0.4]
sampler = fit_gp(initial, data, verbose=True)

Fitting GP
Running burn-in
Running second burn-in
Running production


#### Grafico curvas sampleadas del GP ajustado

In [15]:
print("Making plots")
samples = sampler.flatchain
x = np.linspace(min_time, max_time, 500)

plt.figure()
plt.errorbar(t_obs, y_obs, yerr=err_obs, fmt=".b",ecolor='r', capsize=0)

s = samples[10]
#for s in samples[np.random.randint(len(samples), size=24)]:
for i in xrange(24):
    gp = george.GP(np.exp(s[0]) * kernels.Matern32Kernel(np.exp(s[1])))
    gp.compute(t_obs, err_obs)
    m = gp.sample_conditional(y_obs - model(s[2:], t_obs), x) + model(s[2:], x)
    plt.plot(x, m, color="#4682b4", alpha=0.3)
    #plt.plot(x, m, alpha=0.3)
    
plt.ylabel(r"$y$")
plt.xlabel(r"$t$")
plt.xlim(min_time, max_time)
plt.title("results with Gaussian process noise model")
plt.show()

Making plots


#### Grafico modelo ajustado con su incertidumbre

In [16]:
x = np.linspace(min_time, max_time, 500)

plt.close()
plt.figure()
plt.errorbar(t_obs, y_obs, yerr=err_obs, fmt=".b", ecolor='r', capsize=0)

gp = george.GP(np.exp(s[0]) * kernels.Matern32Kernel(np.exp(s[1])))
gp.compute(t_obs, yerr=err_obs)
mu, cov = gp.predict(y_obs - model(s[2:], t_obs), x)
mu = mu + model(s[2:], x)
#sigma = cov.diagonal()
sigma = np.sqrt(np.diag(cov))

plt.plot(x, mu, color="#4682b4", alpha=0.3)

# Agrego el intervalo de confianza
plt.fill(np.concatenate([x, x[::-1]]), \
        np.concatenate([mu - 1.9600 * sigma,
                       (mu + 1.9600 * sigma)[::-1]]), \
        alpha=.5, fc='#C0C0C0', ec='None', label='95% confidence interval')
plt.show()

Comparo los valores de las features

In [20]:
import os, sys
lib_path = os.path.abspath('../FATS')
sys.path.append(lib_path)

import FATS

In [60]:
feature_names = ['Amplitude', 'Beyond1Std', 'Con','MaxSlope', 'MedianAbsDev','MedianBRP',
                 'PairSlopeTrend', 'Rcs', 'Skew', 'SmallKurtosis','Std','StestonK',
                 'VariabilityIndex', 'meanvariance']

fs = FATS.FeatureSpace(Data=['magnitude', 'time', 'error'],
                       featureList=['Amplitude', 'Beyond1Std', 'Con',
                                    'MaxSlope', 'MedianAbsDev','MedianBRP',
                                    'PairSlopeTrend', 'Rcs', 'Skew', 'SmallKurtosis',
                                    'Std','StestonK', 'VariabilityIndex', 'meanvariance'], excludeList=None)


In [62]:
fs.calculateFeature([y_obs, t_obs, err_obs])

fs = fs.calculateFeature([y_obs, t_obs, err_obs])
print map(lambda x: float("{0:.5f}".format(x)),fs.result(method='dict'))

ValueError: Unknown format code 'f' for object of type 'str'

In [56]:
fs = fs.calculateFeature([y_obs, t_obs, err_obs])
real_values = np.array(fs.result(method=''))

mu, cov = gp.predict(y_obs - model(s[2:], t_obs), t_obs)
mu = mu + model(s[2:], t_obs)

fs = fs.calculateFeature([mu, t_obs, np.sqrt(np.diag(cov))])
model_values = np.array(fs.result(method=''))

percentage = (abs(real_values - model_values) / real_values) * 100

print map(lambda x: float("{0:.2f}".format(x)),percentage)

[0.51, 6.06, 0.0, 90.3, 0.24, 6.45, -0.0, 1.02, 3.92, -28.28, 0.11]


#### Histograma

In [57]:
# Sampleo las muestras a las que les voy a calcular las features
samples = sampler.flatchain
x = np.linspace(min_time, max_time, 500)

muestras = []

for s in samples[np.random.randint(len(samples), size=100)]:
    gp = george.GP(np.exp(s[0]) * kernels.Matern32Kernel(np.exp(s[1])))
    gp.compute(t_obs, err_obs)
    m = gp.sample_conditional(y_obs - model(s[2:], t_obs), t_obs) + model(s[2:], t_obs)
    mu, cov = gp.predict(m, t_obs)

    sigma = np.sqrt(np.diag(cov))
    muestras.append((m, sigma))

In [58]:
# Calculos las features y las guardo en una matriz
feature_samples = []

for m in muestras:
    fs = fs.calculateFeature([m[0], t_obs, m[1]])
    feature_samples.append(fs.result(method=''))

In [59]:
aux = np.array(feature_samples)
aux.shape

(100, 11)

In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
14


In [52]:
import graf

for i in xrange(len(feature_names)):
    #sample_hist(aux[:,i], real_values[i], feature_names[i], 'Resultados/Graficos Ajuste GP/Be_lc/')
    graf.sample_hist(aux[:,i], feature_name=feature_names[i], save_path='Resultados/Graficos Ajuste GP/Be_lc/')

IndexError: index 11 is out of bounds for axis 1 with size 11

In [ ]:
print aux[:,0].mean()
print real_values[0]

In [18]:
s

array([ 4.98225425, -4.9991223 ,  9.45956269,  0.29556945,  1.22432777])